# Quantization Coding

Given a matrix $X$ of any shape, the goal is to get a appriximatly good encoder and decoder, such that

$$X \overset{Encoder}{\longrightarrow\longrightarrow\longrightarrow} code \overset{Decoder}{\longrightarrow\longrightarrow\longrightarrow} X'$$

The motivation of quantization is to compress float data (like model gradients) in a lossless way.

+ Suppose each value in the array $x \in X$ is a 8B(64 bit) float
+ Want to quantize $X$ with $s$-bit
+ $x = sign(x) \times \xi(x) \times \| X \|_2^2$, where $\xi(x)$ is

    + Do normalization based on the 2-norm, $x' = \frac{x}{\|X\|_2^2}$
    
    + Make $2^s$ intervals, corresponding to each value from $[0, 2^s-1]$.
    
    + For each $x'$, find the interval $[l, l+1]$ where it lies.
    
      $0 \le l \le x' \le l+1 \le 2^s -1$
    
    + Random sample a value in the corersponding interval, $v \in [l, l+1]$
    
    + $\begin{eqnarray}
       \xi(x) =
       \begin{cases}
       l, & l \le v \le x'\\
       l+1, & x' \le v \le l+1
       \end{cases}
      \end{eqnarray}$
+ Therefore compress each float $x$ with 2-bit sign value and $s$-bit $\xi$. Each 64-bit variable can include $64/(2 + s)$ float values.

In [1]:
import numpy as np
import torch
import math
import numpy.linalg as LA

In [2]:
def encode(v, **kwargs):
    if isinstance(v, (torch.Tensor, torch.cuda.FloatTensor)):
        w = v.cpu().numpy().flat[:]
    elif isinstance(v, np.ndarray):
        w = v.flat[:]
    else:
        raise ValueError("Object passed to encode not ndarray or torch.Tensor")

    norm = LA.norm(v)

    quantization_level = kwargs['quantization_level']
    s = (1 << quantization_level) - 1
    shape = v.shape
    num_int_each_64_bits = 64 / (2 + quantization_level)
    num_section = num_int_each_64_bits
    len_each_section = (w.shape[0] + num_section - 1) / num_section
    w = np.pad(w, (0, len_each_section*num_section - w.shape[0]), mode='constant')

    sign_array = np.sign(w)
    sign_array += 1
    sign_array = sign_array.astype('uint64')
    normalization_array = np.abs(w) / norm * s

    truncated_array = normalization_array.astype(int)
    prob_array =  normalization_array - truncated_array
    dice_array = np.random.rand(len(prob_array))
    xi_array = truncated_array + (dice_array > prob_array)
    xi_array = xi_array.astype('uint64')
    
    old_sign_array = sign_array
    old_xi_array = xi_array
    
    xi_array = xi_array.reshape((num_section, len_each_section))
    sign_array = sign_array.reshape((num_section, len_each_section))
    
    neo_array = np.zeros(len_each_section)
    neo_array = neo_array.astype('uint64')

    for i in range(num_int_each_64_bits):
        xi = xi_array[i]
        sign = sign_array[i]
        neo_array <<= (2 + quantization_level)
        neo_array = neo_array | (sign << quantization_level | xi)

    code = {'neo': neo_array, 'norm': norm, 'quantization_level': quantization_level,
            'len_each_section': len_each_section, 'num_int_each_64_bits': num_int_each_64_bits,
            'shape': shape}

    return code

In [3]:
def decode(code, cuda=False, implementation='numpy', codes=[], **kwargs):
    if implementation == 'numpy':
        norm = code['norm']
        quantization_level = code['quantization_level']
        s = (1 << quantization_level) - 1
            
        real_size = np.prod(code['shape'])
        
        neo_array = code['neo'].astype('uint64')
        num_int_each_64_bits = code['num_int_each_64_bits']
        num_section = num_int_each_64_bits
        len_each_section = code['len_each_section']
        xi_array = np.ones((num_section, len_each_section))
        sign_array = np.ones((num_section, len_each_section))
        mask_for_xi = (1 << quantization_level) - 1
        mask_for_sign = 3 << quantization_level
        for i in range(num_int_each_64_bits)[::-1]:
            sign_array[i] = (neo_array & mask_for_sign) >> quantization_level
            xi_array[i] = neo_array & mask_for_xi
            neo_array >>= (2 + quantization_level)
        
        xi_array = xi_array.reshape(-1).astype('uint64')
        sign_array = sign_array.reshape(-1).astype('int8')
        sign_array -= 1
        v = sign_array * xi_array * norm / s
        
        v = v[:real_size]
        v = v.reshape(code['shape'])
        
    else:
        raise ValueError('Whoops, implementation')
    return v

In [4]:
test_a = np.random.rand(20, 1)

print 'each size: {}, total size: {}'.format(test_a[0].nbytes, test_a.nbytes)
print
print 'Original array:'
print test_a[:, 0]

each size: 8, total size: 160

Original array:
[ 0.95723658  0.1136488   0.14324333  0.58009869  0.45105075  0.41803557
  0.7058255   0.53538994  0.20890947  0.61961766  0.65952282  0.05665859
  0.43925004  0.52186763  0.73093325  0.77428901  0.61678737  0.61952965
  0.16519608  0.9223033 ]


In [5]:
for quantization_level in range(1, 10):
    print 'Quantization level: {}'.format(quantization_level)
    kwargs = {'quantization_level': quantization_level}
    code = encode(test_a, **kwargs)
    v = decode(code=code)
    print v
    print
    print
    print

Quantization level: 1
[[ 0.        ]
 [ 2.56052405]
 [ 2.56052405]
 [ 2.56052405]
 [ 2.56052405]
 [ 2.56052405]
 [ 0.        ]
 [ 2.56052405]
 [ 2.56052405]
 [ 2.56052405]
 [ 2.56052405]
 [ 2.56052405]
 [ 2.56052405]
 [ 2.56052405]
 [ 2.56052405]
 [ 0.        ]
 [ 2.56052405]
 [ 2.56052405]
 [ 2.56052405]
 [ 2.56052405]]



Quantization level: 2
[[ 1.70701604]
 [ 0.85350802]
 [ 0.85350802]
 [ 0.        ]
 [ 0.85350802]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.85350802]
 [ 0.        ]
 [ 0.        ]
 [ 0.85350802]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.85350802]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 1.70701604]]



Quantization level: 3
[[ 0.7315783 ]
 [ 0.        ]
 [ 0.        ]
 [ 0.36578915]
 [ 0.7315783 ]
 [ 0.7315783 ]
 [ 0.36578915]
 [ 0.7315783 ]
 [ 0.        ]
 [ 0.36578915]
 [ 0.36578915]
 [ 0.36578915]
 [ 0.7315783 ]
 [ 0.36578915]
 [ 0.36578915]
 [ 1.09736745]
 [ 0.36578915]
 [ 0.36578915]
 [ 0.36578915]
 [ 0.7315783 ]]



Quantization level: 4


A simple demo on a random matrix compression.

The precision gets higher as quantization level $s$ becomes larger.